In [1]:
#!pip install nltk

#nltk.download('stopwords');

In [2]:
import nltk
import time
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
import pandas as pd
import re



Load tweets from local: for now we use the lab1 loading for reference

In [3]:
# Load data
csv_file_data = '../Data/Output_v5_data_anonim.csv'
data_raw = pd.read_csv(csv_file_data, quotechar='"',low_memory=False)
data_raw = data_raw[:100000]
#print(data_raw)

csv_file_hashtag = '../Data/Output_v5_hash_anonim.csv'
hashtags_raw =pd.read_csv(csv_file_hashtag, quotechar='"',low_memory=False)


In [4]:
hashtags_raw = hashtags_raw.groupby(['tweet_id'])['hastags'].apply(', '.join).reset_index()

data = data_raw.merge(hashtags_raw, on='tweet_id', how='left')

data = data.sort_values(by=['tweet_id'])

data.rename(columns={'hastags':'hashtags'},inplace=True)
data.index=data.tweet_id
data.head()

,tweet_id,text,user_screen_name,created_at,favorite_count,retweet_count,user_followers,url,user_followers_index,hashtags
tweet_id,,,,,,,,,,
1,1,The polls are absolutely wrong! All you have t...,590ecf4c5b88c57f5658f3097f32efd2,Tue Oct 20 15:11:08 +0000 2020,0,0,3,https://twitter.com/twitter/statuses/131857045...,0.832457,NaN
8,8,_Lai And the dangerous part? When there is pu...,203afe8023cab3bb1fa15ec48c737a71,Tue Oct 20 15:10:48 +0000 2020,18,1,2502,https://twitter.com/twitter/statuses/131857037...,0.915293,NaN
19,19,News flash. tRump IS a politician. https://t.c...,a708a5aef0063b7a40cd2bbbf51984c0,Tue Oct 20 15:10:10 +0000 2020,1,0,275,https://twitter.com/twitter/statuses/131857020...,0.845215,NaN
35,35,#NewYorkPost published a report saying e-mails...,c1170c45d30c5a1c258ba52e5e8349f6,Tue Oct 20 15:09:31 +0000 2020,1,1,106,https://twitter.com/twitter/statuses/131857004...,0.837466,"NewYorkPost, JoeBiden, HunterBiden, BREAKING, ..."
39,39,@realDonaldTrump @JoeBiden GOP HAS APPOINT...,2d9df2e6b9ecba9c119fc674c211b5ec,Tue Oct 20 15:09:14 +0000 2020,0,0,197,https://twitter.com/twitter/statuses/131856997...,0.841707,NaN


Functions:
    - getTerms(line)  --> extract tweet items
    - create_index(lines) --> get dictionaries
    - search(query, index) --> busca docs que contienen todos los elementos de la query

En cuanto tengamos la estructura de los tweets tendremos que modificar getTerms (getTerms(tweets)) y create_index (create_index(tweets)) para que se adapten a la estructura y construyan los diccionarios con lo que queremos.

In [5]:
def getTerms():
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """
    global data
    data['words'] = data['text'].str.lower()

    # Special characters removal (mentions, URLs, punctuation and rt)
    data['words'] = data['words'].apply(lambda x: re.sub(r"([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", x))  
    data['words'] = data['words'].str.split()
    # Stopwords removal
    stop = set(stopwords.words("english"))
    data['words'] = data['words'].apply(lambda x:([y for y in x if y not in stop]))
    # Stemming
    stemming = PorterStemmer()
    data['words'] = data['words'].apply(lambda x: ([stemming.stem(word) for word in x]))


In [6]:
def getTerms_str(line):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """
        
    stemming = PorterStemmer()
    stops = set(stopwords.words("english"))
    ## START CODE
    line = line.lower() ## Transform in lowercase
    line = re.sub(r"([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", line)
    line = line.split() ## Tokenize the text to get a list of terms
    line = [term for term in line if term not in stops]  ##eliminate the stopwords (HINT: use List Comprehension)
    line =[stemming.stem(term) for term in line] ## perform stemming (HINT: use List Comprehension)
    ## END CODE
    return line


In [7]:
def create_index_tfidf():
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    lines -- collection of Wikipedia articles
    numDocuments -- total number of documents
    
    Returns:
    index - the inverted index (implemented through a python dictionary) containing terms as keys and the corresponding 
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    """
    global data

    numDocuments=len(data.index)
    
    index=defaultdict(list)
    tf=defaultdict(list) #term frequencies of terms in documents (documents in the same order as in the main index)
    df=defaultdict(int)         #document frequencies of terms in the corpus
    idf=defaultdict(float)

    for num, row in data.iterrows():

        page_id = row['tweet_id']       
        terms = row['words'] #page_title + page_text
       
        
        ## ===============================================================        
        ## create the index for the **current page** and store it in termdictPage
        ## termdictPage ==> { ‘term1’: [currentdoc, [list of positions]], ...,‘termn’: [currentdoc, [list of positions]]}
        
        ## Example: if the curr_doc has id 1 and his text is 
        ## "web retrieval information retrieval":
        
        ## termdictPage ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}
        
        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        termdictPage={}

        for position, term in enumerate(terms): ## terms contains page_title + page_text
            try:
                # if the term is already in the dict append the position to the corrisponding list
                termdictPage[term][page_id].append(position) 
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                termdictPage[term]=[page_id, array('I',[position])] #'I' indicates unsigned int (int in python)
        
        #normalize term frequencies
        # Compute the denominator to normalize term frequencies (formula 2 above)
        # norm is the same for all terms of a document.
        norm=0
        for term, posting in termdictPage.items(): 
            # posting is a list containing doc_id and the list of positions for current term in current document: 
            # posting ==> [currentdoc, [list of positions]] 
            # you can use it to inferr the frequency of current term.
            norm+=len(posting[1])**2
        norm=math.sqrt(norm)


        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in termdictPage.items():     
            # append the tf for current term (tf = term frequency in current doc/norm)
            tf[term].append(np.round(len(posting[1])/norm,4))  ## SEE formula (1) above
            #increment the document frequency of current term (number of documents containing the current term)
            df[term]= df[term] + 1  # increment df for current term
        
        #merge the current page index with the main index
        for termpage, postingpage in termdictPage.items():
            index[termpage].append(postingpage)
            
    # Compute idf following the formula (3) above. HINT: use np.log
    for term in df:
        idf[term] = np.round(np.log(float(len(termdictPage.items())/df[term])),4)

    return index, tf, df, idf


In [8]:
def rankDocuments(terms, docs, index, idf, tf):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    titleIndex -- mapping between page id and page title
    
    Returns:
    Print the list of ranked documents
    """
    global data
    
    T=1
    # I'm interested only on the element of the docVector corresponding to the query terms 
    # The remaing elements would became 0 when multiplied to the queryVector
    docVectors=defaultdict(lambda: [0]*len(terms)) # I call docVectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary
    queryVector=[0]*len(terms)    

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms) # get the frequency of each term in the query. 
    # Example: collections.Counter(["hello","hello","world"]) --> Counter({'hello': 2, 'world': 1})
    # HINT: use when computing tf for queryVector
    
    query_norm = la.norm(list(query_terms_count.values()))
    
    for termIndex, term in enumerate(terms): #termIndex is the index of the term in the query
        if term not in index:
            continue
                    
        ## Compute tf*idf(normalize tf as done with documents)
        queryVector[termIndex]= query_terms_count[term]/query_norm * idf[term]

        # Generate docVectors for matching docs
        for docIndex, (doc, postings) in enumerate(index[term]):
            # Example of [docIndex, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....
            
            #tf[term][0] will contain the tf of the term "term" in the doc 26            
            if doc in docs:
                docVectors[doc][termIndex]=tf[term][docIndex] * idf[term]  # TODO: check if multiply for idf
                
    # calculate the score of each doc
    # compute the cosine similarity between queyVector and each docVector:
    # HINT: you can use the dot product because in case of normalized vectors it corresponds to the cosine siilarity
    # see np.dot
    method = input('Select Method:\n\t-Standard\n\t-Custom\n')
    if method=='Custom':
        print('Custom Method')
        #Our own score function

        docScores=[ [data.loc[doc,'user_followers_index']*np.dot(curDocVec, queryVector), doc] for doc, curDocVec in docVectors.items() ]
    else:
        print('Standar Method')
        #The  TF-IDF + cosine similarity scoring
        docScores=[ [np.dot(curDocVec, queryVector), doc] for doc, curDocVec in docVectors.items() ]
    docScores.sort(reverse=True)
    resultDocs=[x[1] for x in docScores]
    #print document titles instead if document id's
    
    if len(resultDocs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)    
    #print ('\n'.join(resultDocs), '\n')
    return docScores

In [9]:
def search_tf_idf(query, index):
    '''
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''
    query=getTerms_str(query)
    
    init_docs = [posting[0] for posting in index[query[0]]]
    docs = set(init_docs)
    
    for term in query[1:]:
        try:
            # store in termDocs the ids of the docs that contain "term"                        
            termDocs=[posting[0] for posting in index[term]]
            
            # docs = docs Union termDocs
            docs = docs.intersection(set(termDocs))
        except:
            #term is not in index
            pass
    docs=list(docs)
    ranked_docs = rankDocuments(query, docs, index, idf, tf)   
    return ranked_docs

In [10]:
def print_tweet(tweet, n, score):
    '''
    Print information about tweet n from the ranking in a organized format.
    
    Argument:
    tweet -- information to print
    n -- number of tweet in the ranking
    '''
    print(f'\nResult {n}')
    print(f'\nScore {score}')
    print(f'\nCoeffiecient {tweet.user_followers_index.values[0]}')
    print("-"*70)
    print(tweet.text.values[0])
    print(f'\nTweet by user {tweet.user_screen_name.values[0]}')
    print(f'\t❤️ {tweet.favorite_count.values[0]}\t🔁 {tweet.retweet_count.values[0]}\t📅{tweet.created_at.values[0]}')
    #print(f'Hashtags:')
    print(f'Url: {tweet.url.values[0]}\n')
    print("="*70)

In [11]:
from IPython.display import display
def search_engine():
    query = input("Search: ")
    result = search_tf_idf(query, index)
    scoresx= [x[0] for x in result]
    positionx= [x[1] for x in result]
    ranking = data[data['tweet_id'].isin(positionx)]
    print("="*70)
    for i in range(0,min(20,len(ranking)),1):
        print_tweet(ranking.iloc[[(i)]], i+1,scoresx[i])

# Creating the index and searching

In [12]:
start=time.time()
getTerms()
print(time.time()-start)
data['words'].head()

31.8982150554657


tweet_id
1     [poll, absolut, wrong, watch, ralli, see, incr...
8     [lai, danger, part, pushback, dont, say, anyth...
19                     [news, flash, trump, politician]
35    [newyorkpost, publish, report, say, email, obt...
39    [realdonaldtrump, joebiden, gop, appoint, 14, ...
Name: words, dtype: object

In [13]:
start=time.time()
index, tf, df, idf = create_index_tfidf()
print(time.time()-start)

30.625789403915405


In [14]:
search_engine()

Search: trump idiot
Select Method:
	-Standard
	-Custom
Standar
Standar Method

Result 1

Score 50.17562120887066

Coeffiecient 0.9391708621886696
----------------------------------------------------------------------
Yeah sure you idiot. To disagree with her father would be suicidal for tRump employees. https://t.co/vKkhqYjWQ9

Tweet by user 76d673e0b2c09d74ed777e91823bd8af
	❤️ 3	🔁 0	📅Tue Oct 20 12:34:49 +0000 2020
Url: https://twitter.com/twitter/statuses/1318531113472675840


Result 2

Score 50.17562120887066

Coeffiecient 0.8471878991479893
----------------------------------------------------------------------
Now they wear MASKS !!! Pathetic all of them !!! 220000 our AMERICANS died and counting Trump's dancing &amp; GOPs star wearing MASKS FUCK THEM ALL  they're no longer VALUE to USA  they're nothing but PUTIN'S USEFUL IDIOTS ..! https://t.co/8Mlsr6OJpT

Tweet by user 0cf9e8ab765f294551b2b0627a94e774
	❤️ 0	🔁 0	📅Mon Oct 19 11:30:30 +0000 2020
Url: https://twitter.com/twitter/statu

In [15]:
search_engine()


Search: trump idiot
Select Method:
	-Standard
	-Custom
Custom
Custom Method

Result 1

Score 43.18381783204097

Coeffiecient 0.9391708621886696
----------------------------------------------------------------------
Yeah sure you idiot. To disagree with her father would be suicidal for tRump employees. https://t.co/vKkhqYjWQ9

Tweet by user 76d673e0b2c09d74ed777e91823bd8af
	❤️ 3	🔁 0	📅Tue Oct 20 12:34:49 +0000 2020
Url: https://twitter.com/twitter/statuses/1318531113472675840


Result 2

Score 43.145175896964304

Coeffiecient 0.8471878991479893
----------------------------------------------------------------------
Now they wear MASKS !!! Pathetic all of them !!! 220000 our AMERICANS died and counting Trump's dancing &amp; GOPs star wearing MASKS FUCK THEM ALL  they're no longer VALUE to USA  they're nothing but PUTIN'S USEFUL IDIOTS ..! https://t.co/8Mlsr6OJpT

Tweet by user 0cf9e8ab765f294551b2b0627a94e774
	❤️ 0	🔁 0	📅Mon Oct 19 11:30:30 +0000 2020
Url: https://twitter.com/twitter/status